![Image Description](https://img00.deviantart.net/3390/i/2015/092/3/f/ultimate_gaming_wallpaper_w_w_w_by_cristyann-d8o3801.jpg)

**Dataset Features:**

**Game Information:**
Titles of the games, release dates, and information about the publishers and developers involved in each title's creation.

**Sales and Revenue:**
Tracks game sales performance, including the total number of copies sold, revenue generated, and original price details.

**Player Engagement Metrics:**
Provides insights into player behavior through metrics like average playtime and peak player counts, helping to measure user engagement and retention.

**Reviews and Scores:**
Aggregated review scores and ratings from both users and critics, enabling a detailed view of public reception and critical acclaim.

**Dynamic Market Insights:**
Sales rank trends, pricing fluctuations, and market performance data over time, giving a holistic view of game performance across the year.

**Objective of the Notebook:** This notebook aims to extract meaningful insights from the data, providing analysis and visualizations on key metrics such as game sales, revenue generation, user engagement, and review trends. The goal is to uncover patterns in the gaming market and help identify factors that drive a game’s success on the Steam platform.

By the end of this notebook, you will have a deep understanding of the dataset's structure, key trends in the gaming industry, and actionable insights for game developers, data analysts, and researchers alike.

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tanent_id/oauth2/token"}

dbutils.fs.mount(
source = "abfss://gaming-dataset@gamingdataset202409.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/games",
extra_configs = configs)




True

In [0]:
%fs
ls "/mnt/games"

path,name,size,modificationTime
dbfs:/mnt/games/Raw-data/,Raw-data/,0,1727238629000
dbfs:/mnt/games/Trans-data/,Trans-data/,0,1727238644000


In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import col, row_number, avg, year, sum, to_date, month, when, count, round
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType


In [0]:
# Define schema for the gaming dataset
GamingDataset_Schema = StructType([
    StructField("name", StringType()),                 # Game name
    StructField("releaseDate", StringType()),          # Release date
    StructField("copiesSold", IntegerType()),          # Total copies sold (using Integer to store the total count)
    StructField("price", DoubleType()),                # Price (Double for currency values)
    StructField("revenue", DoubleType()),              # Revenue (Double for money values)
    StructField("avgPlaytime", DoubleType()),          # Average playtime (Double for playtime in hours)
    StructField("reviewScore", DoubleType()),          # Review score (Double for ratings, which could have decimals)
    StructField("publisherClass", StringType()),       # Publisher classification (AAA, AA, Indie)
    StructField("publishers", StringType()),           # Publisher name
    StructField("developers", StringType()),           # Developer name(s)
    StructField("steamId", StringType())               # Steam unique identifier
])


In [0]:
games = spark.read.format("csv") \
    .option("header", "true") \
    .option("quote", '"').option("escape", '"').option("multiLine", "true").schema(GamingDataset_Schema).load("/mnt/games/Raw-data/games")

In [0]:
games.display()


name,releaseDate,copiesSold,price,revenue,avgPlaytime,reviewScore,publisherClass,publishers,developers,steamId
WWE 2K24,07-03-2024,165301,99.99,8055097.0,42.36514031444467,71.0,AAA,2K,Visual Concepts,2315690
EARTH DEFENSE FORCE 6,25-07-2024,159806,59.99,7882151.0,29.65106126155342,57.0,Indie,D3PUBLISHER,SANDLOT,2291060
Sins of a Solar Empire II,15-08-2024,214192,49.99,7815247.0,12.45259326556514,88.0,Indie,Stardock Entertainment,"Ironclad Games Corporation,Stardock Entertainment",1575940
Legend of Mortal,14-06-2024,440998,19.99,7756399.0,24.79781729089117,76.0,Indie,"Paras Games,Obb Studio Inc.",Obb Studio Inc.,1859910
Shin Megami Tensei V: Vengeance,13-06-2024,141306,59.99,7629252.0,34.25849627863547,96.0,AA,SEGA,ATLUS,1875830
Soulmask,30-05-2024,304685,29.99,7540563.0,95.69781345051696,79.0,Indie,Qooland Games,CampFire Studio,2646460
The Universim,22-01-2024,328945,29.99,7534369.0,16.86853054897363,81.0,Indie,Crytivo,Crytivo,352720
Bellwright,23-04-2024,280955,29.99,7481940.0,41.418884797624166,78.0,Indie,"Donkey Crew,Snail Games USA",Donkey Crew,1812450
Night of the Dead,31-05-2024,394364,29.99,7091792.0,27.59707192847936,79.0,Indie,Jackto Studios,Jackto Studios,1377380
Empires of the Undergrowth,07-06-2024,408292,29.99,6950952.0,38.60212424802546,95.0,AA,"Hooded Horse,Slug Disco",Slug Disco,463530


##1. Sales and Revenue Analysis


#### **a. Top-Selling Games Over Time (Monthly/Yearly)**

Aggregates total copies sold for each game by release year, highlighting the top-selling titles across years.

In [0]:
games = games.withColumn("releaseDate", to_date(col("releaseDate"), "dd-MM-yyyy"))

games.groupBy('name', month('releaseDate').alias('release_month')) \
    .agg(sum('copiesSold').alias('total_copies_sold')) \
    .orderBy('total_copies_sold', ascending=False) \
    .limit(10).display()

name,release_month,total_copies_sold
Banana,4,30739148
Palworld,1,16704850
Black Myth: Wukong,8,15517278
HELLDIVERS™ 2,2,11905198
7 Days to Die,7,9877443
Sons Of The Forest,2,8693478
Once Human,7,5709407
V Rising,5,4784609
The First Descendant,6,4043850
Chained Together,6,3946801


####b. Revenue per Copy Sold for Different Price Points
Calculates the average revenue per copy at different price points, offering insights into price vs. profitability.

In [0]:
games.groupBy('publishers','name','price') \
  .agg((sum('revenue') / sum('copiesSold')).alias('revenue_per_copy')) \
  .orderBy('revenue_per_copy', ascending=False) \
  .display()

publishers,name,price,revenue_per_copy
PERFECT WORLD GAMES,Perfect World M,0.0,219.15233545647558
Chinesegamer,吞食天地2：誕生Reborn,0.0,90.86248182249757
Starlight Games,Der Königsruf,0.0,89.10422535211268
2K,NBA 2K25,69.99,62.99099623436563
Electronic Arts,EA SPORTS™ Madden NFL 25,69.99,62.99099384414402
"CAPCOM Co., Ltd.",Dragon's Dogma 2,69.99,62.45184445286012
Bandai Namco Entertainment,TEKKEN 8,69.99,61.63570711098892
Electronic Arts,F1® 24,69.99,58.332900118294226
SEGA,Like a Dragon: Infinite Wealth,69.99,57.57783012482062
SEGA,Persona 3 Reload,69.99,56.48583196207491


Databricks data profile. Run in Databricks to view.

####c. Revenue Growth Over Time for Top Publishers
Tracks yearly revenue growth of the top 5 publishers, revealing market trends and dominance.

In [0]:
windowSpec = Window.orderBy(col('total_revenue').desc())

top_publishers_df = games.groupBy('publishers') \
                      .agg(sum('revenue').alias('total_revenue')) \
                      .withColumn('rank', row_number().over(windowSpec)) \
                      .filter(col('rank') <= 5)

games.join(top_publishers_df, 'publishers') \
  .groupBy('publishers', month('releaseDate').alias('release_month')) \
  .agg(sum('revenue').alias('monthly_revenue')) \
  .orderBy('publishers', 'release_month') \
  .display()

publishers,release_month,monthly_revenue
"CAPCOM Co., Ltd.",1,3465018.0
"CAPCOM Co., Ltd.",3,1.11478291E8
"CAPCOM Co., Ltd.",6,683951.0
"CAPCOM Co., Ltd.",7,1276574.0
"CAPCOM Co., Ltd.",9,977191.0
Game Science,8,8.37793356E8
Newnight,2,2.17017892E8
PlayStation Publishing LLC,2,4.35635596E8
PlayStation Publishing LLC,3,2.2958942E7
PlayStation Publishing LLC,5,4.9948315E7


##2. Release Date and Playtime Insights

####a. Average Playtime by Game Release Year

Shows average game playtime based on release year, highlighting which years had the most engaging games.

In [0]:
games.groupBy('name') \
  .agg(avg('avgPlaytime').alias('avg_playtime')) \
  .orderBy('avg_playtime', ascending=False) \
  .display()

name,avg_playtime
Firestone: Online Idle RPG,296.3328524525871
洪荒：我挂机成圣,271.6146279284957
江湖客栈-The Jianghu,260.3405090672399
Farmer Against Potatoes Idle,256.3350027720608
吞食天地2：誕生Reborn,239.43979712743936
Idle Hero TD - Tower Defense,189.05701139969932
Space industrial empire,185.9252736
Destiny Online,128.7942781499453
Grim Clicker,122.75703853781792
Summoners War,112.85633956732288


####b. Playtime Trends Across Different Publisher Classes Over Time
Compares playtime trends across different publisher types over time.


In [0]:
games.groupBy('publisherClass', year('releaseDate').alias('release_year')) \
  .agg(avg('avgPlaytime').alias('avg_playtime')) \
  .orderBy('release_year', 'avg_playtime', ascending=False) \
  .display()

publisherClass,release_year,avg_playtime
AAA,2024,16.399648371205657
AA,2024,15.760170250857406
Indie,2024,12.053672220961833
Hobbyist,2024,8.461323747170796


####c. Top Games by Playtime That Also Have High Sales (Playtime-Sales Correlation)
Identifies games with above-average playtime and strong sales, reflecting commercial and engagement success.

In [0]:
games.filter((col('avgPlaytime') > games.select(avg('avgPlaytime')).first()[0]) &
          (col('copiesSold') > games.select(avg('copiesSold')).first()[0])) \
  .orderBy('avgPlaytime', 'copiesSold', ascending=False) \
  .limit(10) \
  .display()

name,releaseDate,copiesSold,price,revenue,avgPlaytime,reviewScore,publisherClass,publishers,developers,steamId
Firestone: Online Idle RPG,2024-04-01,335093,0.0,1.5689364148647271E7,296.3328524525871,79.0,Indie,Holyday Studios,Holyday Studios,1013320
Farmer Against Potatoes Idle,2024-09-05,234948,0.0,1153910.9084421117,256.3350027720608,96.0,Indie,Oni Gaming,Oni Gaming,1535560
Grim Clicker,2024-01-30,274957,0.0,355113.9777668873,122.75703853781792,80.0,Indie,"EvilCharm Games,Aleksandr Golovkin",EvilCharm Games,1160750
Summoners War,2024-01-07,540801,0.0,1.0787342E7,112.85633956732288,0.0,Indie,Com2uS,Com2uS,2426960
Predecessor,2024-08-20,433605,0.0,4523839.944318563,109.91382768034087,79.0,Indie,Omeda Studios,Omeda Studios,961200
Soulmask,2024-05-30,304685,29.99,7540563.0,95.69781345051696,79.0,Indie,Qooland Games,CampFire Studio,2646460
Workers & Resources: Soviet Republic,2024-06-20,456555,39.99,9363299.0,91.46189909283493,93.0,AA,Hooded Horse,3Division,784150
7 Days to Die,2024-07-25,9877443,44.99,8.9781931E7,85.91358519475969,89.0,AA,The Fun Pimps Entertainment LLC,The Fun Pimps,251570
TEKKEN 8,2024-01-25,734722,69.99,4.528511E7,60.82372173009936,71.0,AAA,Bandai Namco Entertainment,Bandai Namco Studios Inc.,1778820
Cultivation Tales,2024-04-17,201079,19.99,2443731.0,60.02369541089477,37.0,Indie,Ac Games,Ac Games,1504570


##3. Publisher and Developer Insights

####a. Revenue Concentration for Top 5 Publishers vs Others
Compares revenue of the top 5 publishers against all others to show market share concentration.

In [0]:
top_publishers_df = games.groupBy('publishers') \
                      .agg(sum('revenue').alias('total_revenue')) \
                      .orderBy('total_revenue', ascending=False) \
                      .limit(5)

games.withColumn('publisher_group', 
              when(col('publishers').isin([row.publishers for row in top_publishers_df.collect()]), col('publishers')).otherwise('Others')) \
  .groupBy('publisher_group') \
  .agg(sum('revenue').alias('total_revenue')) \
  .orderBy('total_revenue', ascending=False) \
  .display()

publisher_group,total_revenue
Others,1.874666949029264E9
Game Science,8.37793356E8
PlayStation Publishing LLC,5.08885199E8
Pocketpair,3.92328553E8
Newnight,2.17017892E8
"CAPCOM Co., Ltd.",1.17881025E8


####b. Success Rate of Publishers Based on Review Scores and Sales
Calculates the success rate of publishers based on high review scores and sales thresholds.

In [0]:
success_df = games.groupBy('publishers') \
  .agg(sum(when((col('reviewScore') > 80) & (col('copiesSold') > 500000), 1).otherwise(0)).alias('successful_games'),
       count('*').alias('total_games'))

success_df.withColumn('success_rate', (col('successful_games') / col('total_games')) * 100) \
  .orderBy('success_rate', ascending=False) \
  .display()

publishers,successful_games,total_games,success_rate
WarpFrog,1,1,100.0
Newnight,1,1,100.0
Playstack,2,2,100.0
Anegar Games,1,1,100.0
Nokta Games,1,1,100.0
Landfall,1,1,100.0
Oddshot Games,1,1,100.0
Sky,1,1,100.0
"Red Nexus Games Inc.,IndieArk",1,1,100.0
Temple Gates Games,1,1,100.0


##4. Review Score and Player Sentiment

####a. Games with High Reviews but Low Sales (Potential Missed Opportunities)
Finds highly rated games with low sales, indicating missed market opportunities.

In [0]:
games.filter((col('reviewScore') > 80) & 
          (col('copiesSold') < games.select(avg('copiesSold')).first()[0])) \
  .orderBy(['reviewScore', 'copiesSold'], ascending=[False, True]) \
  .display()

name,releaseDate,copiesSold,price,revenue,avgPlaytime,reviewScore,publisherClass,publishers,developers,steamId
Lilja and Natsuka Painting Lies,2024-07-24,1085,22.99,22449.0,8.273406759307356,100.0,Indie,"Frontwing USA,Bushiroad Inc.",Frontwing,2840900
Oji-Mama/憧れのガチムチおじさんがボクだけのドスケベママになっちゃった❤,2024-05-23,1138,25.0,25605.0,1.589665166121501,100.0,Indie,SKSK団,SKSK団,2725330
Speed Crew,2024-01-31,1695,19.99,26802.0,3.066085176262451,100.0,Indie,Wild Fields,Wild Fields,2367480
TRAMCITY HAKODATE,2024-01-04,1888,14.99,25471.0,2.946587843328908,100.0,Indie,TRAMWORKS,TRAMWORKS,2614000
Kitten Burst,2024-01-20,2073,24.99,40690.0,7.522090444087317,100.0,Indie,Lithodelphis,Lithodelphis,1592360
Natsu-Mon: 20th Century Summer Kid,2024-06-26,2115,39.99,73248.0,5.826172175775137,100.0,AA,"Spike Chunsoft Co., Ltd.","TOYBOX Inc.,Millennium Kitchen Co., Ltd.",2839280
Dünnes Eis - Das Spiel zum Song,2024-05-16,6564,3.99,23571.0,0.9798791592039828,100.0,Indie,"Lorke Records, Inh. Anne Baumann","Baumann Bergmann Pokinsson,Jan Baumann",2961990
Mirage Feathers,2024-09-06,6859,4.99,27717.0,1.1706268898128227,100.0,Indie,oyasumi Workshop,oyasumi Workshop,2719060
DEVIL BLADE REBOOT,2024-05-23,9966,15.99,123743.0,4.484006580046724,100.0,Indie,SHIGATAKE GAMES,SHIGATAKE GAMES,2882440
Our Adventurer Guild,2024-04-12,23699,14.99,284501.0,39.46176078042124,100.0,Indie,GreenGuy,GreenGuy,2026000


####b. Review Score Discrepancy by Publisher Class
Compares average review scores across publisher types.

In [0]:
games.groupBy('publisherClass') \
  .agg(avg('reviewScore').alias('avg_review_score')) \
  .orderBy('avg_review_score', ascending=False) \
  .display()

publisherClass,avg_review_score
Indie,76.57340507302075
AA,74.17808219178082
AAA,72.75
Hobbyist,67.0


####c. Average Review Score vs Sales vs Playtime (Multi-factor Analysis)
Analyzes games excelling in review scores, sales, and playtime across all metrics.

In [0]:
games.filter((col('reviewScore') > games.select(avg('reviewScore')).first()[0]) & 
          (col('copiesSold') > games.select(avg('copiesSold')).first()[0]) &
          (col('avgPlaytime') > games.select(avg('avgPlaytime')).first()[0])) \
  .orderBy('reviewScore', 'copiesSold', 'avgPlaytime', ascending=False) \
  .display()

name,releaseDate,copiesSold,price,revenue,avgPlaytime,reviewScore,publisherClass,publishers,developers,steamId
shapez 2,2024-08-15,261028,24.99,4810868.0,16.307715687349233,99.0,Indie,"tobspr Games,Gamera Games",tobspr Games,2162800
Fields of Mistria,2024-08-05,265456,13.99,3342356.0,15.649349536238686,98.0,Indie,NPC Studio,NPC Studio,2142790
Blade and Sorcery,2024-06-17,1606175,29.99,3.1266686E7,19.72078698069728,97.0,Indie,WarpFrog,WarpFrog,629730
Balatro,2024-02-20,1602797,14.99,2.047921E7,32.72552809973843,97.0,Indie,Playstack,LocalThunk,2379780
The Planet Crafter,2024-04-10,1381487,23.99,2.1286732E7,28.17299459798124,97.0,Indie,Miju Games,Miju Games,1284190
Rusty's Retirement,2024-04-26,278210,6.99,1585093.0,36.76470154540171,97.0,Indie,Mister Morris Games,Mister Morris Games,2666510
Rabbit and Steel,2024-05-09,213267,14.99,2771018.0,14.014176071536836,97.0,Indie,mino_dev,mino_dev,2132850
Black Myth: Wukong,2024-08-19,15517278,59.99,8.37793356E8,20.065040850375343,96.0,AAA,Game Science,Game Science,2358720
Abiotic Factor,2024-05-02,520841,24.99,1.0501115E7,18.452732943062458,96.0,Indie,Playstack,Deep Field Games,427410
Nova Drift,2024-08-12,445361,17.99,4047331.0,24.72868457429413,96.0,Indie,Pixeljam,Chimeric,858210


##5. Price Elasticity and Revenue Optimization

####a. Price Impact on Sales Across Different Price Brackets
Analyzes how different price brackets impact average sales and revenue.

In [0]:
games.withColumn('price_range', 
              when(col('price') < 10, 'Low Price (< $10)')
              .when((col('price') >= 10) & (col('price') <= 30), 'Medium Price ($10-$30)')
              .otherwise('High Price (> $30)')) \
  .groupBy('price_range') \
  .agg(avg('copiesSold').alias('avg_copies_sold'), avg('revenue').alias('avg_revenue')) \
  .orderBy('avg_revenue', ascending=False) \
  .display()

price_range,avg_copies_sold,avg_revenue
High Price (> $30),446383.2027972028,1.5973340727272727E7
Medium Price ($10-$30),81419.5918128655,1540782.505263158
Low Price (< $10),156926.90438247012,691267.34667184


####b. Price Sensitivity by Publisher Class
Examines price sensitivity and sales performance across various publisher types.

In [0]:
games.groupBy('publisherClass',month('releaseDate').alias('release_month'), 'price') \
  .agg(round(avg('copiesSold'),1).alias('avg_copies_sold')) \
  .orderBy('publisherClass', 'price') \
  .display()

publisherClass,release_month,price,avg_copies_sold
AA,1,0.0,286872.0
AA,6,0.0,4043850.0
AA,7,0.0,239148.0
AA,1,2.99,11023.0
AA,7,4.99,12613.0
AA,4,4.99,48759.0
AA,5,4.99,25501.0
AA,7,6.99,3966.0
AA,8,6.99,13896.0
AA,4,7.99,1448054.0
